In [26]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [27]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch
import pickle


In [28]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [29]:
sats_games = []
PREFIXES = ['oct04_encumberedheuristic']
# PREFIXES = ['sep20']
for prefix in PREFIXES:
    sats_games += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    
sats_games = list(set(sats_games))

sats_games = [
        s for s in sats_games if ('undersell_allowed' in s) or ('tie_break' in s) or (s.endswith('base') or s.endswith('risk_averse') or s.endswith('spite')) or ('no_activity' in s) or ('grace_period' in s)
]

sats_games = [s for s in sats_games if 'activity' not in s]


print(len(sats_games))
print(sats_games)

60
['oct04_encumberedheuristic/oct04_encumberedheuristic_4_risk_averse_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedheuristic_0_base_tie_break', 'oct04_encumberedheuristic/oct04_encumberedheuristic_0_risk_averse_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedheuristic_4_risk_averse_tie_break', 'oct04_encumberedheuristic/oct04_encumberedheuristic_4_base_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedheuristic_2_base_tie_break', 'oct04_encumberedheuristic/oct04_encumberedheuristic_1_risk_averse_grace_period', 'oct04_encumberedheuristic/oct04_encumberedheuristic_2_base_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedheuristic_1_risk_averse_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedheuristic_3_base_tie_break', 'oct04_encumberedheuristic/oct04_encumberedheuristic_1_base_tie_break', 'oct04_encumberedheuristic/oct04_encumberedheuristic_4_spite_undersell_allowed', 'oct04_encumberedheuristic/oct04_encumberedhe

# For running the real dumb test case

In [18]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

60


In [19]:
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 0.5)
# for config in pgrid:
#     CONFIG = 'cfr_init_2' # The YML network config folder for the main jobs
#     N_SEEDS = 1 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun24_cfr_gamut_7', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [30]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 6)
for config in pgrid:
    CONFIG = 'cfr_port_10_ext' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 5400 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 50 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='nov21_external_fixed_v4_no_other_heuristics', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 105776
Submitted batch job 105777
Submitted batch job 105778
Submitted batch job 105779
Submitted batch job 105780
Submitted batch job 105781
Submitted batch job 105782
Submitted batch job 105783
Submitted batch job 105784
Submitted batch job 105785
Submitted batch job 105786
Submitted batch job 105787
Submitted batch job 105788
Submitted batch job 105789
Submitted batch job 105790
Submitted batch job 105791
Submitted batch job 105792
Submitted batch job 105793
Submitted batch job 105794
Submitted batch job 105795
Submitted batch job 105796
Submitted batch job 105797
Submitted batch job 105798
Submitted batch job 105799
Submitted batch job 105800
Submitted batch job 105801
Submitted batch job 105802
Submitted batch job 105803
Submitted batch job 105804
Submitted batch job 105805
Submitted batch job 105806
Submitted batch job 105807
Submitted batch job 105808
Submitted batch job 105809
Submitted batch job 105810
Submitted batch job 105811
Submitted batch job 105812
S

In [23]:
!squeue --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)
            105296 oct16_encumbereddeviations_oct16_encumbereddeviations_4_spite_grace_period-cfr_port_10_ext  R .     21:39     1 ip-10-255-1-184
            105297 oct16_encumbereddeviations_oct16_encumbereddeviations_4_risk_averse_tie_break-cfr_port_10_  R .     21:39     1 ip-10-255-1-184
            105298 oct16_encumbereddeviations_oct16_encumbereddeviations_1_base-cfr_port_10_extexternal_plus_  R .     21:39     1 ip-10-255-1-188
            105299 oct16_encumbereddeviations_oct16_encumbereddeviations_0_base_tie_break-cfr_port_10_extexte  R .     21:39     1 ip-10-255-1-188
            105300 oct16_encumbereddeviations_oct16_encumbereddeviations_0_risk_averse_grace_period-cfr_port_  R .     21:39     1 ip-10-255-1-192
            105301 oct16_encumbereddeviations_oct16_encumbereddeviations_4_spite_tie_break-cfr_port_10_extext  R .   

In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [ ]:
check_queue()

In [24]:
!squeue | wc -l

241


In [ ]:
!squeue | grep "may30" | wc -l

In [7]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)